In [ ]:
# Config
# import torch
import os
# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Monitoring
from tqdm.notebook import tqdm
# IO
# from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob
# Parallel processing
from joblib import Parallel, delayed
# import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO


In [17]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [ ]:
conf=0.001 ## confidence threshold
type = "obb"
task = "obb"
iou=0.33 ## iou threshold

In [ ]:
model_configs = [
    # {
    #     # Example Config
    #     # west_bengal -> haryana
    #     'base_state' : "west_bengal",
    #     'target_state' : "haryana",
    #     'model_name' : "yolo11m-obb.pt",
    #     'gt_image_dir' : "",
    #     'gt_label_dir' : "",
    #     'class_count' : "10_120_1000",
    #     'model_path' : "",
    #     'image_size' : 640,
    #     'epochs' : 100,
    # }
    {
        # Example Config
        'base_state' : "delhi thera",
        'target_state' : "delhi thera",
        'model_name' : "yolo11m-obb.pt",
        'gt_image_dir' : "/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/data/thera_rdn_pro/delhi_ncr_4x/images",
        'gt_label_dir' : "/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/data/thera_rdn_pro/delhi_ncr_4x/labels",
        'class_count' : "10_120_1000",
        'model_path' : "",
        'image_size' : 640,
        'epochs' : 100,
    }
]

In [ ]:
# chosen_index = 

df_configs = pd.DataFrame(model_configs, columns=model_configs[0].keys())
display(df_configs)

# Ask user to choose a config by its index
try:
    chosen_index = int(input("Enter the index of the config you want to use: "))
except ValueError:
    print("Invalid input. Defaulting to first config (index 0).")
    chosen_index = 0

# Check if the chosen index is within the valid range
if chosen_index < 0 or chosen_index >= len(model_configs):
    print("Index out of range. Defaulting to first config (index 0).")
    chosen_index = 0

model_config = model_configs[chosen_index]
print(f"Using configuration: {model_config}")


,base_state,target_state,model_name,gt_image_dir,gt_label_dir,class_count,model_path,image_size,epochs
0,west_bengal,haryana,yolo11m-obb.pt,,,10_120_1000,,640,100


Using configuration: {'base_state': 'west_bengal', 'target_state': 'haryana', 'model_name': 'yolo11m-obb.pt', 'gt_image_dir': '', 'gt_label_dir': '', 'class_count': '10_120_1000', 'model_path': '', 'image_size': 640, 'epochs': 100}


In [ ]:
home_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/"
yolo_runs_dir = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/{type}/labels/"
print(f"Home Directory: {home_dir}")
print(f"YOLO Runs Directory: {yolo_runs_dir}")

base_state = model_config['base_state']
target_state = model_config['target_state']
model_name = model_config['model_name']
gt_image_dir = model_config['gt_image_dir']
gt_label_dir = model_config['gt_label_dir']
class_count = model_config['class_count']
model_path = model_config['model_path']
image_size = model_config['image_size']
epochs = model_config['epochs']

# Print configuration parameters in a formatted way
print("\nModel Configuration:")
print(f"  Base State   : {base_state}")
print(f"  Target State : {target_state}")
print(f"  Model Name   : {model_name}")
print(f"  GT Image Dir : {gt_image_dir}")
print(f"  GT Label Dir : {gt_label_dir}")
print(f"  Class Count  : {class_count}")
print(f"  Model Path   : {model_path}")
print(f"  Image Size   : {image_size}")
print(f"  Epochs       : {epochs}")

result_dir = f"{yolo_runs_dir}train_{base_state}_test_{target_state}_{class_count}_{task}_v2_{model_name}_{image_size}_128_{epochs}_{conf}_{iou}/"
prediction_dir = os.path.join(result_dir, "labels")

print(f"\nResult Directory:   {result_dir}")
print(f"Prediction Directory: {prediction_dir}")

Home Directory: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/
YOLO Runs Directory: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/
Model Configuration:
  Base State   : west_bengal
  Target State : haryana
  Model Name   : yolo11m-obb.pt
  GT Image Dir : ../data/region_performance/test_bihar_same_class_count_10_120_1000/images
  GT Label Dir : ../data/region_performance/test_bihar_same_class_count_10_120_1000/labels
  Class Count  : 10_120_1000
  Model Path   : /home/umang.shikarvar/Punjab-Bihar/Synthetic_Punjab_YOLO.pt
  Image Size   : 640
  Epochs       : 100

Result Directory:   /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_west_bengal_test_haryana_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/
Prediction Directory: /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_west_bengal_test_haryana_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/labels


In [8]:
## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

print(f"\nData YML Path: {data_yml_path}")



Data YML Path: /tmp/tmpknzd8_ak/data.yml


In [9]:
## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path)
print(sv_dataset)
print(len(sv_dataset))
sv_dataset.classes

0


['CFCBK', 'FCBK', 'Zigzag']

In [ ]:
## Yolo model
model = YOLO(model_path)

targets=[]
predictions=[]
for name,_,gt_detection in tqdm(sv_dataset):
    file_name=os.path.splitext(os.path.basename(name))[0]
    # prediction_path=os.path.join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=640,iou=0.33,conf=0.01,exist_ok=True,save_txt=True,max_det=300,name=result_dir, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)

0it [00:00, ?it/s]

In [ ]:
## mAP calculation (non-class agnostic)
mAP_metric = MeanAveragePrecision(class_agnostic=False)
mAP_result=mAP_metric.update(predictions,targets).compute()
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")

# Extract mAP values
mAP_50_95 = mAP_result.map50_95  # mAP 50:95
mAP_50 = mAP_result.map50  # mAP 50
mAP_75 = mAP_result.map75  # mAP 75
print(f"mAP 50:95: {mAP_50_95}, mAP 50: {mAP_50}, mAP 75: {mAP_75}")

# Extract class-wise mAP values
num_classes=3
final_class_wise_mAP = [0]*num_classes
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP

# Compute class-agnostic mAP
mAP_metric_agnostic = MeanAveragePrecision(class_agnostic=True)
mAP_result_agnostic = mAP_metric_agnostic.update(predictions, targets).compute()
# Extract class-agnostic mAP values
mAP_50_95_agnostic = mAP_result_agnostic.map50_95  # mAP 50:95
mAP_50_agnostic = mAP_result_agnostic.map50  # mAP 50
mAP_75_agnostic = mAP_result_agnostic.map75  # mAP 75
print(f"CA mAP 50:95: {mAP_50_95_agnostic}, CA mAP 50: {mAP_50_agnostic}, CA mAP 75: {mAP_75_agnostic}")

Matched classes: []
class_wise_mAP: [0, 0, 0]
class_wise_mAP: []
class_agnostic_result: []


In [ ]:
# Calculate weighted mAP
weights = pd.Series(np.concatenate)
weights = pd.Series(np.concatenate([target.class_id for target in targets])).value_counts()
weight_sum = (1/weights).values.sum()
weight_mapping = {"CFCBK": 1/weights.get(0, 0), "FCBK": 1/weights.get(1, 0), "Zigzag": 1/weights.get(2, 0)}
print(f" Mapping: {weight_mapping}")
print(f"Weight Sum: {weight_sum}")

weighted_mAP = np.sum([weight_mapping[cls]*mAP for cls, mAP in zip(matched_classes, class_wise_mAP)]) / weight_sum
print(f"Weighted mAP: {weighted_mAP}")

In [ ]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "class_count", "Target State", "Epochs"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "Weighted mAP",
                                   "mAP 50:95", "mAP 50", "mAP 75", 
                                   "CA mAP 50:95", "CA mAP 50", "CA mAP 75"], 
                                   index=index)

result_df.loc[(base_state, class_count, target_state, epochs), :] = [f"{x:.6f}" for x in final_class_wise_mAP + [weighted_mAP, mAP_50_95, mAP_50, mAP_75, mAP_50_95_agnostic, mAP_50_agnostic, mAP_75_agnostic]]
display(result_df)

In [ ]:
# output_csv_path = f"{home_dir}result_csv/different_model_performance_cyclegan.csv"
output_csv_path = 

In [ ]:
append_to_csv = False
if append_to_csv:
    if os.path.exists(output_csv_path):
        result_df.to_csv(output_csv_path, mode='a', header=False)
        print(f"Results appended to {output_csv_path}")
    else:
        result_df.to_csv(output_csv_path, header=True)
        print(f"Results saved to {output_csv_path}")
    

In [ ]:
mAP_df = pd.read_csv(output_csv_path)
display(mAP_df)

In [ ]:
display(mAP_df.style.background_gradient(cmap='viridis', axis=0))